In [ ]:
!mkdir -p taxi_raw
!mkdir -p data/taxi_clean

In [ ]:
!wget -nc -O data/taxi_raw/yellow_201601.csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv
!wget -nc -O data/taxi_raw/yellow_201602.csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-02.csv
!wget -nc -O data/taxi_raw/yellow_201603.csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-03.csv
!wget -nc -O data/taxi_raw/yellow_201604.csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-04.csv
!wget -nc -O data/taxi_raw/yellow_201605.csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-05.csv

In [ ]:
import os

import geopandas as gpd
import pandas as pd
pd.options.display.max_columns = None
import matplotlib
%matplotlib inline
import shapely.geometry


In [ ]:
cols = ["tpep_pickup_datetime", "tpep_dropoff_datetime", 
        "pickup_longitude", "pickup_latitude", 
        "dropoff_longitude", "dropoff_latitude",
        "trip_distance"]
boroughs = gpd.read_file("data/nybb_18a/nybb.shp")
manhattan = boroughs[boroughs.BoroName == "Manhattan"].to_crs(epsg=4326)

def process_month(month):
    print "processing month {}".format(month)
    raw_fname = "data/taxi_raw/yellow_2016{}.csv".format("0" + str(month))
    df = pd.read_csv(raw_fname)[cols]
    print "raw df has {} rows".format(len(df))
    before_5am = df[df["tpep_pickup_datetime"].str.split(" ", 1).apply(lambda x: int(x[1].split(":")[0])) < 5]
    before_5am["pickup_point"] = before_5am[["pickup_longitude", "pickup_latitude"]].apply(shapely.geometry.Point, axis=1)
    before_5am["dropoff_point"] = before_5am[["dropoff_longitude", "dropoff_latitude"]].apply(shapely.geometry.Point, axis=1)
    before_5am = before_5am.reset_index(drop=True)
    print "before 5am has {} rows".format(len(before_5am))
    before_5am["geometry"] = before_5am["pickup_point"]
    before_5am = gpd.GeoDataFrame(before_5am)
    before_filter_pickup = gpd.sjoin(before_5am, manhattan, how='inner', op='within')
    del before_filter_pickup["index_right"]
    before_filter_pickup["geometry"] = before_filter_pickup["dropoff_point"]
    before_filter_dropoff = gpd.sjoin(before_filter_pickup, manhattan, how='inner', op='within')
    before_filter_dropoff = before_filter_dropoff[cols]
    before_filter_dropoff.to_pickle("data/taxi_clean/2016{}_filtered.pkl".format("0" + str(month)))
    print "filtered df has {} rows".format(len(before_filter_dropoff))
    del before_filter_dropoff
    del before_filter_pickup
    del before_5am
    del df
    print "======"
    return "OK"

for month in range(1, 6):
    process_month(month)

## Part 2: restart notebook before running this!

In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
from shapely.ops import nearest_points

In [2]:
p = pd.read_pickle("201601_filtered.pkl")
p = gpd.GeoDataFrame(p)
nodes = gpd.read_file("data/mn_nodes.shp").to_crs(epsg=4326)

In [3]:
p["pickup_point"] = p[["pickup_longitude", "pickup_latitude"]].apply(shapely.geometry.Point, axis=1)
p["dropoff_point"] = p[["dropoff_longitude", "dropoff_latitude"]].apply(shapely.geometry.Point, axis=1)


In [4]:
p["geometry"] = p["pickup_point"].apply(lambda x: x.buffer(.0005))

In [ ]:
pts = nodes.geometry.unary_union
i = 0
def near(row, pts=pts):
    global i
    i = i + 1
    if i % 100 == 0:
        print "{} out of {}".format(i, len(p))
    point = row.geometry
    n = nearest_points(point, pts)
    nearest = nodes.geometry == n[0]
    return nodes[nearest].NODEID.get_values()[0]
p["nearest_origin"] = p.apply(lambda row: near(row), axis=1)

In [6]:
joined = gpd.sjoin(p, nodes, how='inner', op='contains')

In [13]:
del joined["VIntersect"]
del joined["index_righ"]
del joined["BoroCode"]
del joined["BoroName"]
del joined["Shape_Leng"]

In [26]:
import numpy as np
i = 0
def closest(g):
    if len(g) == 1:
        return g.iloc[0]
    global i
    i = i + 1
    if i % 1000 == 0:
        print i
    s = g.pickup_point.iloc[0]
    nod = nodes.loc[g.index_right].geometry
    distances = [s.distance(s2) for s2 in nod]
    return g.iloc[np.argmin(distances)]

closest_only = joined.groupby(joined.index).apply(closest)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

KeyboardInterrupt: 